In [1]:
import numpy as np
import pandas as pd
    
import json
import requests
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

import folium
import matplotlib.pyplot as plt

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# Load html script from URL and create soup object

url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641"
#url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050." 
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html = urlopen(url)
soup = BeautifulSoup(html,'lxml')

## Extract the table data from html and create a dataframe

In [3]:
rows = soup.find_all('tr')
headers = rows[0].find_all('th')


list_rows = []
for row in rows:
    if len(row.find_all('td')) == 3:
        row_td = row.find_all('td')
        headers = rows[0].find_all('th')
    
        for i,v in enumerate(headers):
            cleantext = BeautifulSoup(str(v),"lxml").get_text()
            headers[i] = cleantext.rstrip('\n')
        
        for i,v in enumerate(row_td):
            cleantext = BeautifulSoup(str(v),"lxml").get_text()
            row_td[i] = cleantext.rstrip('\n')
        
        list_rows.append(row_td)
        
df = pd.DataFrame(list_rows,columns=headers)
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


## Correct rows where [Borough] or [Neighborhood] are empty

In [4]:
# Remove rows where Borough is unassigned
df['Borough'].replace('Not assigned',float("NaN"),inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West
207,M8Z,Etobicoke,Mimico NW
208,M8Z,Etobicoke,The Queensway West
209,M8Z,Etobicoke,Royal York South West


In [5]:
# Check to see if any Neighborhood cells are empty and correct them with borough name
for i,n in enumerate(df['Neighbourhood'] == ""):
    if n is True:
        print(str(i) + ": " + df['Neighbourhood'].iloc[i] + " , " + df['Borough'].iloc[i])

## Construct a new DataFrame 

Goal here is to only have unique postal code rows with neighborhoods appended together

In [6]:
boroughs = []
neighborhoods = []
codes = df['Postcode'].unique()

for i,code in enumerate(codes):
    #print(code)
    #print(df['Borough'].iloc[i])
    boroughs.append(df['Borough'].iloc[i])
    post_group = df.groupby(['Postcode']).get_group(code)
    hoods = post_group[['Neighbourhood']].values

    hood_list = ""
    for n in hoods:
        hood_list += n + ", "
    hood_list = hood_list[0].rstrip(', ')
    #print(hood_list)
    #print()
    boroughs[0].rstrip(", ")
    
    neighborhoods.append(hood_list)
    

new_dic = {"PostalCode" : codes,"Borough" : boroughs, "Neighborhood" : neighborhoods}

df_final = pd.DataFrame(new_dic)
df_final

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,Downtown Toronto,"Lawrence Heights, Lawrence Manor"
4,M7A,North York,Not assigned
...,...,...,...
98,M8X,North York,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,East Toronto,Church and Wellesley
100,M7Y,North York,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,North York,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [7]:
num_data = np.shape(df_final)[0]
print("%d rows of data" % num_data)

103 rows of data


## Add location data

In [11]:
# Read csv data into a dataframe
geo = pd.read_csv('Geospatial_coordinates.csv')
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [14]:
# Sort neighborhood information by PostalCode before merging with coordinates
df_final.sort_values('PostalCode',inplace=True)
df_final.reset_index(drop=True,inplace=True)
df_final

df_final[['Latitude','Longitude']] = geo[['Latitude','Longitude']]
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Queen's Park,"Rouge, Malvern",43.806686,-79.194353
1,M1C,East York,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Etobicoke,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,East York,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,West Toronto,Weston,43.706876,-79.518188
99,M9P,North York,Westmount,43.696319,-79.532242
100,M9R,West Toronto,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,North York,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
